In [1]:
## imports
import numpy as np
import cvxpy as cp
import pandas as pd
import matplotlib.pyplot as plt
import itertools
%run ccpp_light.ipynb
from IPython.display import clear_output
%matplotlib inline
%config InlineBackend.figure_format='retina'
clear_output()

### data

In [2]:
dic_results = {'UB':[],'LB':[],'N':[],'repID':[],'time':[],'certif':[],\
               'd':[],'tau':[],'p':[],'skip_':[]}

In [3]:
''' dimensions & hyper-parameters'''

np.random.seed(2308)

d_list = [int(2)]
skip_list = [False,True]
N = int(5e2)
tau_list =  [.05,.15,.25]
N_reps = int(5)
otilde = 1
o = 2
TL = 500
MT_cert = 60

''' commands '''

xi_inf_bound = 5
x_inf_bound = 10
granularity = 6
frac = 23/50

''' elements  '''
def rnd_pos(p):
    buf = np.zeros(p)
    id_pos = np.random.randint(0,p)
    buf[id_pos]= np.random.choice(np.linspace(-xi_inf_bound,xi_inf_bound,granularity))
    denom = np.random.randint(1,100,p-1)
    buf[np.setdiff1d(np.arange(p),[id_pos])] = xi_inf_bound*np.random.randint(-denom,denom+1,p-1)/denom
    return buf


# from INRIA 
def proj_simplex_(vec, R=1,tol=1e-3):
    """ 
    References
    ----------
    [1] Efficient Projections onto the .1-Ball for Learning in High Dimensions
        John Duchi, Shai Shalev-Shwartz, Yoram Singer, and Tushar Chandra.
        International Conference on Machine Learning (ICML 2008)
        http://www.cs.berkeley.edu/~jduchi/projects/DuchiSiShCh08.pdf
    """
    assert R > 0, "radius R must be strictly positive (%d <= 0)" % R
    d, = vec.shape  # will raise ValueError if vec is not 1-D
    if abs(vec.sum() - R)<=tol and np.alltrue(vec >= 0):
        return vec
    sorted_vec = np.sort(vec)[::-1]
    cssv = np.cumsum(sorted_vec)
    buf = np.nonzero(sorted_vec * np.arange(1, d+1) > (cssv - R))[0][-1]
    shift = float(cssv[buf] - R) / (1+buf)
    w = (vec - shift).clip(min=0)
    return w

## ball proximity based
def proj_ball_(x,xi_embd,nm_power,R,tol=5e-4):
    dxxi_embd = x-xi_embd
    abs_dxxi_embd = np.abs(dxxi_embd)
    if nm_power==1:
        if np.sum(abs_dxxi_embd)<=R+tol:
            return x
        return xi_embd + proj_simplex_(abs_dxxi_embd,R)*np.sign(dxxi_embd)
    elif nm_power==np.inf:
        if max(abs_dxxi_embd)<=R+tol:
            return x
        return xi_embd + np.minimum(R,np.maximum(-R,dxxi_embd))
    elif nm_power==2:
        buf = (np.sum(abs_dxxi_embd**2))**(1/2)
        if buf<=R+tol:
            return x
        return xi_embd + dxxi_embd*R/max(R,buf)
    else:
        return None

In [ ]:
for d in d_list:

    print('====> d = '+str(d))
    print(' ')

    p = d
    
    R_base = 2*d*xi_inf_bound*frac

    x_ref = np.random.choice([-xi_inf_bound,xi_inf_bound],d) + ( (np.arange(d))**2 )/d

    ## numpy set-up ##
    if o==2:
        def obj_val(x):
            return np.sum((x-x_ref)**2)
    else:
        def obj_val(x):
            return np.linalg.norm(x-x_ref,o)
            
    ## cvxpy set-up ##
    x_cvx = cp.Variable(d)
    xi_param = cp.Parameter(p)
    big_bound = max(np.max(np.abs(x_ref)),xi_inf_bound+R_base) # big enough so that min_x in X of F(x) s.t. x in R(s) is the same as min_x of F(x) s.t. x in R(s)
    X = [x_cvx>=-big_bound,x_cvx<=big_bound]
    
    if o==2:
        obj_ = cp.sum_squares(x_cvx-x_ref)
    else:
        obj_ = cp.norm(x_cvx-x_ref,o)

    def cstr_gen_cvxpy(xi_embd,nm_power=otilde,R=R_base):
        return cp.norm(x_cvx-xi_embd,nm_power)-R

    
    if otilde==1:
        
        if p==2:
            c_bound_cvxpy = [x_cvx[0]-xi_param[0]+x_cvx[1]-xi_param[1],\
                            -x_cvx[0]+xi_param[0]+x_cvx[1]-xi_param[1],\
                            x_cvx[0]-xi_param[0]-x_cvx[1]+xi_param[1],\
                            -x_cvx[0]+xi_param[0]-x_cvx[1]+xi_param[1]]
        elif p==3:
            c_bound_cvxpy = [x_cvx[0]-xi_param[0]+x_cvx[1]-xi_param[1]+x_cvx[2]-xi_param[2],\
                            -x_cvx[0]+xi_param[0]+x_cvx[1]-xi_param[1]+x_cvx[2]-xi_param[2],\
                            x_cvx[0]-xi_param[0]-x_cvx[1]+xi_param[1]+x_cvx[2]-xi_param[2],\
                            -x_cvx[0]+xi_param[0]-x_cvx[1]+xi_param[1]+x_cvx[2]-xi_param[2],\
                            x_cvx[0]-xi_param[0]+x_cvx[1]-xi_param[1]-x_cvx[2]+xi_param[2],\
                            -x_cvx[0]+xi_param[0]+x_cvx[1]-xi_param[1]-x_cvx[2]+xi_param[2],\
                            x_cvx[0]-xi_param[0]-x_cvx[1]+xi_param[1]-x_cvx[2]+xi_param[2],\
                            -x_cvx[0]+xi_param[0]-x_cvx[1]+xi_param[1]-x_cvx[2]+xi_param[2]]
    elif otilde==np.inf:
        
        if p==2:
            c_bound_cvxpy = [cp.max([x_cvx[0]-xi_param[0],x_cvx[1]-xi_param[1]]),\
                            cp.max([-x_cvx[0]+xi_param[0],x_cvx[1]-xi_param[1]]),\
                            cp.max([x_cvx[0]-xi_param[0],-x_cvx[1]+xi_param[1]]),\
                            cp.max([-x_cvx[0]+xi_param[0],-x_cvx[1]+xi_param[1]])]
        elif p==3:
            c_bound_cvxpy = [cp.max([x_cvx[0]-xi_param[0],x_cvx[1]-xi_param[1],x_cvx[2]-xi_param[2]]),\
                            cp.max([-x_cvx[0]+xi_param[0],x_cvx[1]-xi_param[1],x_cvx[2]-xi_param[2]]),\
                            cp.max([x_cvx[0]-xi_param[0],-x_cvx[1]+xi_param[1],x_cvx[2]-xi_param[2]]),\
                            cp.max([-x_cvx[0]+xi_param[0],-x_cvx[1]+xi_param[1],x_cvx[2]-xi_param[2]]),\
                            cp.max([x_cvx[0]-xi_param[0],x_cvx[1]-xi_param[1],-x_cvx[2]+xi_param[2]]),\
                            cp.max([-x_cvx[0]+xi_param[0],x_cvx[1]-xi_param[1],-x_cvx[2]+xi_param[2]]),\
                            cp.max([x_cvx[0]-xi_param[0],-x_cvx[1]+xi_param[1],-x_cvx[2]+xi_param[2]]),\
                            cp.max([-x_cvx[0]+xi_param[0],-x_cvx[1]+xi_param[1],-x_cvx[2]+xi_param[2]])]
            
    else:
        
        print('not yet ')

    ################

    def single_s_specialized(xi_s):
        x_s = proj_ball_(x_ref,xi_s,otilde,R_base)
        return obj_val(x_s),[x_s]


    for repID in range(N_reps):

        print(' ')
        print('rep #'+str(repID+1))
        print(' ')

        # xi data generation
        xi_list = np.array([rnd_pos(int(p)) for s in range(N)])
        pis = np.ones(N)/N

        for tau in tau_list:

                print('tau = '+str(tau))
                print(' ')

                for sk in skip_list:

                    print('skip_presolve: '+str(sk))


                    prob = Problem(obj_,X,tau,xi_list,pis,cstr_gen=cstr_gen_cvxpy,\
                                 c_bound=c_bound_cvxpy,rel_tol=1e-2,used_solver=cp.MOSEK,individual_nus_comp=single_s_specialized)
                    

                    stat,timing,_,_,_,_ = prob.solve(include_cuts=False,verbose_general=True,T_max=TL,skip_presolve=sk,\
                                      verbose_certify=True,verbose_prune=True,display=False,max_time_cert=MT_cert,max_rounds=1)

                    dic_results['UB'].append(prob.UB)
                    dic_results['LB'].append(prob.LB)
                    dic_results['N'].append(N)
                    dic_results['repID'].append(repID)
                    dic_results['time'].append(timing)
                    dic_results['certif'].append(stat)
                    dic_results['d'].append(d)
                    dic_results['tau'].append(tau)
                    dic_results['p'].append(p)
                    dic_results['skip_'].append(sk)
                    print(' ')

                    df = pd.DataFrame(data=dic_results)
                    df.to_csv('benchmark_final_p_'+str(p)+'_o_'+str(o)+'_otilde_'+str(otilde)+'.csv')
                

====> d = 2
 
 
rep #1
 
tau = 0.05
 
skip_presolve: False
 
pre-pass oracle...
LB = 0.0 | self.UB = 53.213986831496875 | pre-pruned: 4
 
 
empirical cstr (LB): 0.9040000000000004
current lower-bound: 35.84222241829322
 
 
empirical cstr (UB): 0.9500000000000004
current upper-bound: 53.213986831496875
 
 
trial @6 | remaining checks [%] -> 97.72
trial @22 | remaining checks [%] -> 88.63
trial @61 | remaining checks [%] -> 79.54
trial @126 | remaining checks [%] -> 70.45
trial @191 | remaining checks [%] -> 61.36
trial @229 | remaining checks [%] -> 52.27
trial @283 | remaining checks [%] -> 43.18
trial @357 | remaining checks [%] -> 34.09
trial @376 | remaining checks [%] -> 25.0
trial @409 | remaining checks [%] -> 15.90
trial @473 | remaining checks [%] -> 6.818
 
after optimality scan: 0 indices were added to the pruned set
 
 
 /!\ pre_solve termination /!\ 
safe indices: 294 ; pruned indices: 4
best feasible solution:: 53.213986831496875
                                     CVXPY 

(CVXPY) Mar 08 01:33:39 PM: 7871     7486     4634     38       3.9604669614e+03     4.2084500165e+01     98.94       42.3  
(CVXPY) Mar 08 01:33:40 PM: 8319     7800     4546     52       3.9604669614e+03     4.2275768998e+01     98.93       44.1  
(CVXPY) Mar 08 01:33:42 PM: 8699     8121     4484     63       3.9604669614e+03     4.2461851513e+01     98.93       45.7  
(CVXPY) Mar 08 01:33:44 PM: 9089     8441     4498     75       3.7223174174e+03     4.2547196691e+01     98.86       47.3  
(CVXPY) Mar 08 01:33:45 PM: 9501     8759     4470     85       2.1248648845e+03     4.3072604022e+01     97.97       49.1  
(CVXPY) Mar 08 01:33:47 PM: 9957     9070     4330     89       2.1248648845e+03     4.3244919380e+01     97.96       51.1  
(CVXPY) Mar 08 01:33:50 PM: 10343    9381     4254     19       2.0166399192e+03     4.3244996022e+01     97.86       53.5  
(CVXPY) Mar 08 01:33:52 PM: 10741    9693     4186     28       2.0166399192e+03     4.3265382259e+01     97.85       55.6  


(CVXPY) Mar 08 01:35:33 PM: 
(CVXPY) Mar 08 01:35:33 PM: 
(CVXPY) Mar 08 01:35:33 PM: Integer solution solution summary
(CVXPY) Mar 08 01:35:33 PM:   Problem status  : PRIMAL_FEASIBLE
(CVXPY) Mar 08 01:35:33 PM:   Solution status : INTEGER_OPTIMAL
(CVXPY) Mar 08 01:35:33 PM:   Primal.  obj: 4.9444807280e+01    nrm: 5e+01    Viol.  con: 8e-10    var: 0e+00    cones: 0e+00    itg: 0e+00  
-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
(CVXPY) Mar 08 01:35:33 PM: Problem status: optimal
(CVXPY) Mar 08 01:35:33 PM: Optimal value: 4.944e+01
(CVXPY) Mar 08 01:35:33 PM: Compilation took 1.497e+00 seconds
(CVXPY) Mar 08 01:35:33 PM: Solver (including time spent in interface) took 1.570e+02 seconds
final target accuracy:: 0.0
total running time: 235.04889273643494 [s]
cost:: 49.44488760015554
 
skip_preso

(CVXPY) Mar 08 01:36:25 PM: 2431     2449     2432     205      8.0053875214e+04     8.0549692385e+00     99.99       23.0  
(CVXPY) Mar 08 01:36:28 PM: 2742     2760     2743     231      8.0053875214e+04     8.0549692385e+00     99.99       25.9  
(CVXPY) Mar 08 01:36:30 PM: 3053     3071     3054     257      8.0053875214e+04     8.0549692385e+00     99.99       28.2  
(CVXPY) Mar 08 01:36:32 PM: 3365     3383     3366     284      8.0053875214e+04     8.0549692385e+00     99.99       30.5  
(CVXPY) Mar 08 01:36:35 PM: 3676     3694     3677     310      8.0053875214e+04     8.0549692385e+00     99.99       33.6  
(CVXPY) Mar 08 01:36:38 PM: 3984     4002     3985     336      8.0053875214e+04     8.0549692385e+00     99.99       36.5  
(CVXPY) Mar 08 01:36:41 PM: 4287     4305     4288     362      8.0053875214e+04     8.0549692385e+00     99.99       39.0  
(CVXPY) Mar 08 01:36:43 PM: 4590     4608     4591     387      8.0053875214e+04     8.0549692385e+00     99.99       41.7  


(CVXPY) Mar 08 01:40:01 PM: 22148    22142    8527     153      1.1298142080e+04     3.6459229766e+01     99.68       239.6 
(CVXPY) Mar 08 01:40:05 PM: 22446    22440    8485     178      1.1298142080e+04     3.6459229766e+01     99.68       243.6 
(CVXPY) Mar 08 01:40:09 PM: 22740    22734    8449     203      7.9922118786e+03     3.6459229766e+01     99.54       247.5 
(CVXPY) Mar 08 01:40:13 PM: 23020    23014    8413     226      7.9922118786e+03     3.6459229766e+01     99.54       250.9 
(CVXPY) Mar 08 01:40:16 PM: 23316    23310    8367     251      7.9922118786e+03     3.6459229766e+01     99.54       253.8 
(CVXPY) Mar 08 01:40:18 PM: 23614    23608    8335     276      7.9922118786e+03     3.6459229766e+01     99.54       256.4 
(CVXPY) Mar 08 01:40:21 PM: 23913    23906    8332     301      7.9922118786e+03     3.6459229766e+01     99.54       258.9 
(CVXPY) Mar 08 01:40:23 PM: 24213    24205    8306     326      7.9922118786e+03     3.6459229766e+01     99.54       261.5 


...

(CVXPY) Mar 08 03:03:11 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> MOSEK
(CVXPY) Mar 08 03:03:11 PM: Applying reduction Dcp2Cone
(CVXPY) Mar 08 03:03:11 PM: Applying reduction CvxAttr2Constr
(CVXPY) Mar 08 03:03:12 PM: Applying reduction ConeMatrixStuffing
(CVXPY) Mar 08 03:03:12 PM: Applying reduction MOSEK
(CVXPY) Mar 08 03:03:13 PM: Finished problem compilation (took 1.537e+00 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Mar 08 03:03:13 PM: Invoking solver MOSEK  to obtain a solution.


(CVXPY) Mar 08 03:03:13 PM: Problem
(CVXPY) Mar 08 03:03:13 PM:   Name                   :                 
(CVXPY) Mar 08 03:03:13 PM:   Objective sense        : minimize        
(CVXPY) Mar 08 03:03:13 PM:   Type                   : CONIC (conic optimization p

(CVXPY) Mar 08 03:04:57 PM: 11661    11642    11544    27       7.5426131290e+04     -1.3191450554e-07    100.00      104.7 
(CVXPY) Mar 08 03:05:00 PM: 11941    11922    11824    54       7.5426131290e+04     -1.3191450554e-07    100.00      107.6 
(CVXPY) Mar 08 03:05:03 PM: 12198    12179    12081    79       7.5426131290e+04     -1.3191450554e-07    100.00      110.7 
(CVXPY) Mar 08 03:05:06 PM: 12478    12459    12361    106      7.5426131290e+04     -1.3191450554e-07    100.00      113.5 
(CVXPY) Mar 08 03:05:09 PM: 12766    12747    12649    133      7.5426131290e+04     -1.3191450554e-07    100.00      116.2 
(CVXPY) Mar 08 03:05:12 PM: 13045    13026    12928    159      7.5426131290e+04     -1.3191450554e-07    100.00      118.9 
(CVXPY) Mar 08 03:05:14 PM: 13328    13309    13211    186      7.5426131290e+04     -1.3191450554e-07    100.00      121.6 
(CVXPY) Mar 08 03:05:17 PM: 13609    13590    13492    213      7.5426131290e+04     -1.3191450554e-07    100.00      124.1 


(CVXPY) Mar 08 03:07:56 PM: 30305    30249    24428    74       1.0048030434e+04     -1.3191450554e-07    100.00      283.7 
(CVXPY) Mar 08 03:07:59 PM: 30596    30540    24471    147      1.0048030434e+04     -1.3191450554e-07    100.00      286.2 
(CVXPY) Mar 08 03:08:02 PM: 30886    30830    24545    131      1.0048030434e+04     -1.3191450554e-07    100.00      288.9 
(CVXPY) Mar 08 03:08:04 PM: 31176    31120    24613    149      1.0048030434e+04     -1.3191450554e-07    100.00      291.6 
(CVXPY) Mar 08 03:08:07 PM: 31461    31405    24714    175      1.0048030434e+04     -1.3191450554e-07    100.00      294.2 
(CVXPY) Mar 08 03:08:09 PM: 31740    31684    24783    200      1.0048029863e+04     -1.3191450554e-07    100.00      296.8 
(CVXPY) Mar 08 03:08:12 PM: 32035    31979    24854    227      1.0048029863e+04     -1.3191450554e-07    100.00      299.4 
(CVXPY) Mar 08 03:08:15 PM: 32325    32269    24960    254      1.0048029863e+04     -1.3191450554e-07    100.00      302.0 


(CVXPY) Mar 08 03:10:56 PM: 49430    49341    37833    92       6.3311327832e+03     -1.3191450554e-07    100.00      463.4 
(CVXPY) Mar 08 03:11:00 PM: 49718    49629    37891    115      6.3311327832e+03     -1.3191450554e-07    100.00      467.7 
(CVXPY) Mar 08 03:11:06 PM: 50009    49920    37938    84       6.3311327832e+03     -1.3191450554e-07    100.00      472.9 
(CVXPY) Mar 08 03:11:11 PM: 50296    50207    38005    82       6.3311327832e+03     -1.3191450554e-07    100.00      478.0 
(CVXPY) Mar 08 03:11:11 PM: 
(CVXPY) Mar 08 03:11:11 PM: Objective of best integer solution : 6.331132783201e+03      
(CVXPY) Mar 08 03:11:11 PM: Best objective bound               : -1.319145055428e-07     
(CVXPY) Mar 08 03:11:11 PM: Initial feasible solution objective: Undefined
(CVXPY) Mar 08 03:11:11 PM: Construct solution objective       : Not employed
(CVXPY) Mar 08 03:11:11 PM: User objective cut value           : Not employed
(CVXPY) Mar 08 03:11:11 PM: Number of cuts generated        

(CVXPY) Mar 08 03:12:50 PM: 580      589      563      49       5.0894311069e+04     3.9394077043e+01     99.92       4.6   
(CVXPY) Mar 08 03:12:53 PM: 881      873      788      11       2.5437675292e+04     3.9394077043e+01     99.85       7.3   
